In [1]:
import sys
import numpy as np
from yolo import YOLO
from PIL import Image
import cv2
import numpy as np
import collections

Using TensorFlow backend.


In [2]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("/home/user/桌面/basketball/Mask_RCNN-master/")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)


class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'mrcnn_mask_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [5]:
def main(yolo):
    img = '/home/user/桌面/1.png'
    image = Image.open(img)
    image_cv = cv2.cvtColor(np.asarray(image),cv2.COLOR_RGB2BGR)  
     
        
    results = model.detect([image_cv], verbose=1)
    r = results[0]
    image_class = r['class_ids'] 
    image_mask = r['masks']
    image_rois = r['rois']
    
    person_box = []
    avg_list=[]
    image_id = list(image_class)
    
    for i in range(len(image_id)):
        predicted_class = image_id[i]
        if predicted_class != 1:
            continue
        mask_box = image_mask[...,:,i]
        image_hls = cv2.cvtColor(image_cv,cv2.COLOR_BGR2HLS)
        image_hls2 = image_hls[...,:,1]
        person_box = mask_box*image_hls2
        rois_box = image_rois[i]
        rois_list = list(rois_box)
        crop_person = person_box[rois_list[0]:rois_list[2],rois_list[1]:rois_list[3]]
        crop_color = image_cv[rois_list[0]:rois_list[2],rois_list[1]:rois_list[3]]
        h,w=crop_person.shape
        a = int(h/8)
        b = int(h*4/5)
        c = int(w/8)
        d = int(w*4/5)
        
        crop_img= crop_person[a:b,c:d]
        hls_1d = crop_img.ravel()
        hls_list = list(hls_1d)
            
        for i in hls_list: 
            if hls_list[i] == 0:
                hls_list.remove(0)
            else: continue
        count = len(hls_list)
        hls = sum(hls_list)
        hls_avg = int(hls/count)
        avg_list.append(hls_avg)
            
            
        print(hls_avg, avg_list)
        cv2.imshow(str(hls_avg),crop_color)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

In [6]:
if __name__ == '__main__':
    main(YOLO())

model_data/yolo.h5 model, anchors, and classes loaded.
Processing 1 images
image                    shape: (277, 764, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
44 [44]
162 [44, 162]
39 [44, 162, 39]
83 [44, 162, 39, 83]
41 [44, 162, 39, 83, 41]
47 [44, 162, 39, 83, 41, 47]
